# Single Run

In [1]:
import pandas as pd

from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master

from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary

from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_32224/317628422.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Task: pazienti storici

In [2]:
historical_data_df = pd.read_csv("../data/historical_data.csv")

patient_provider = PatientsFromHistoricalDataProvider(
                 historical_data= historical_data_df
                 )

In [3]:
or_master_schedule = pd.read_csv("../data/master_schedule_input.csv", sep = ';')
master_schedule  = Master(name = "my master", table=or_master_schedule)

In [4]:
num_of_patients = 100

In [5]:
patients, training = patient_provider.provide_sets(quantity=num_of_patients, quantity_training=1000)


In [6]:
model = NGBLogNormal(description="test", patients=training)
patients = model.predict(patients)

In [7]:

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= num_of_patients,
            robustness_risk= 0.2,
            robustness_overtime= 30,
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
            )

task.patients = patients
task.master_schedule = master_schedule


### Implementor - Adversary

In [8]:
implementor_adversary = ImplementorAdversary(task = task, implementor=StandardImplementor(), adversary=None, description = 'Test')

In [9]:
implementor_adversary.create_instance()

In [10]:
solution_implementor_adversary = implementor_adversary.run(15)

implementor
adversary


In [11]:
for pat in (solution_implementor_adversary._blocks[0].patients):
    print(pat)

Patient id: 5906 
 equipe: D 
 urgency: 4 
 nominal duration: 49 
 days waiting: 169
Patient id: 3714 
 equipe: D 
 urgency: 3 
 nominal duration: 46 
 days waiting: 145
Patient id: 3883 
 equipe: D 
 urgency: 3 
 nominal duration: 46 
 days waiting: 146
Patient id: 6300 
 equipe: D 
 urgency: 4 
 nominal duration: 73 
 days waiting: 329
Patient id: 6198 
 equipe: D 
 urgency: 2 
 nominal duration: 41 
 days waiting: 42
Patient id: 7496 
 equipe: D 
 urgency: 2 
 nominal duration: 40 
 days waiting: 33


solution._blocks[4].patients[0].uncertainty_profile.param_s

### Chance constraint

In [12]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [13]:
chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [14]:
chance_constraints.create_instance()

In [15]:
solution_chance_constraints = chance_constraints.run()

In [16]:
for pat in (solution_chance_constraints._blocks[0].patients):
    print(pat)

AttributeError: 'ConcreteModel' object has no attribute '_blocks'

### Budget Set

In [ ]:
from surgeryschedulingunderuncertainty.optimizer import  BudgetSet
from surgeryschedulingunderuncertainty.implementor import  BudgetSetImplementor


In [ ]:
task.gamma_variation = 0


budget_set = BudgetSet(task = task, 
                              implementor=BudgetSetImplementor(task = task), 
                              description = 'Test')






In [ ]:
soulution_budget_set = budget_set.run()

6
['C', 'D']
----
6
['D', 'C']
----
4
['I']
----
5
['H', 'F']
----
6
['D', 'C']
----
4
['L']
----
6
['C', 'D']
----
5
['F', 'H']
----
2
['I']
----
9
['H', 'F']
----
2
['L']
----
7
['B', 'E', 'A', 'G']
----
7
['E', 'A', 'G', 'B']
----
5
['F', 'H']
----
7
['A', 'B', 'E', 'G']
----
7
['G', 'B', 'E', 'A']
----


In [ ]:
budget_set._instance.get(None).get('time_increment')

{1: 3.7498030910937326,
 2: 4.09161270629753,
 3: 3.6771076447125863,
 4: 12.666269750463933,
 5: 12.892810500396733,
 6: 3.8191502687188916,
 7: 4.084713198016788,
 8: 26.995653104038183,
 9: 7.4679030682143015,
 10: 3.679958286966773,
 11: 3.6086629092750644,
 12: 4.102741409424607,
 13: 4.733383663703206,
 14: 10.713692648253076,
 15: 3.992007088791223,
 16: 3.489027646393643,
 17: 5.205845055324746,
 18: 6.845018641646168,
 19: 5.806914555282409,
 20: 3.5579944452596233,
 21: 4.0308912340708565,
 22: 4.810345213802426,
 23: 6.568997089910099,
 24: 8.3703269285588,
 25: 3.454124784961164,
 26: 8.573698889956118,
 27: 6.525031866843483,
 28: 6.737139575490656,
 29: 6.911171397916507,
 30: 5.2749028865729946,
 31: 9.06606943543722,
 32: 7.396420874422615,
 33: 7.906850052728971,
 34: 4.411391315893926,
 35: 3.6770993766656765,
 36: 9.51165893836611,
 37: 4.968861703785849,
 38: 13.938784874873814,
 39: 4.819762144578456,
 40: 4.330064766628166,
 41: 5.373158346668775,
 42: 12.72091514

In [ ]:
budget_set._instance.get(None).get('gamma')

{1: 6,
 2: 6,
 3: 4,
 4: 5,
 5: 6,
 6: 4,
 7: 6,
 8: 5,
 9: 2,
 10: 9,
 11: 2,
 12: 7,
 13: 7,
 14: 5,
 15: 7,
 16: 7,
 17: 6,
 18: 6,
 19: 4,
 20: 5,
 21: 6,
 22: 4,
 23: 6,
 24: 5,
 25: 2,
 26: 9,
 27: 2,
 28: 7,
 29: 7,
 30: 5,
 31: 7,
 32: 7}

In [ ]:
for block in soulution_budget_set._blocks:
    print(block)
    for pat in (block.patients):
        print(pat)
    print('---')

Patient id: 6196 
 equipe: D 
 urgency: 3 
 nominal duration: 45 
 days waiting: 92
Patient id: 3203 
 equipe: D 
 urgency: 4 
 nominal duration: 46 
 days waiting: 298
Patient id: 1821 
 equipe: D 
 urgency: 3 
 nominal duration: 44 
 days waiting: 147
Patient id: 7199 
 equipe: D 
 urgency: 4 
 nominal duration: 45 
 days waiting: 207
Patient id: 8274 
 equipe: D 
 urgency: 4 
 nominal duration: 45 
 days waiting: 183
---
Patient id: 7064 
 equipe: D 
 urgency: 3 
 nominal duration: 63 
 days waiting: 123
Patient id: 7906 
 equipe: D 
 urgency: 2 
 nominal duration: 45 
 days waiting: 38
Patient id: 4221 
 equipe: D 
 urgency: 2 
 nominal duration: 45 
 days waiting: 24
Patient id: 7613 
 equipe: D 
 urgency: 3 
 nominal duration: 68 
 days waiting: 171
Patient id: 1016 
 equipe: D 
 urgency: 2 
 nominal duration: 45 
 days waiting: 14
---
Patient id: 7119 
 equipe: I 
 urgency: 1 
 nominal duration: 108 
 days waiting: 24
Patient id: 584 
 equipe: I 
 urgency: 3 
 nominal duration: 